# Hands On - Fighting Foodwaste - Introduction to Forecasting

# Import Data

In [ ]:
import pandas as pd
bakery = pd.read_csv("https://raw.githubusercontent.com/casbdai/datasets/main/bakery_sales.csv")

## Check Structure of Data

In [ ]:
bakery.info()

In [ ]:
bakery.head()

## Importing Data correctly

Pandas has very good functionalities for dealing with time series - they save a tremendous lot of pre-processing work.

In [ ]:
bakery = pd.read_csv("https://raw.githubusercontent.com/casbdai/datasets/main/bakery_sales.csv", index_col="Date", parse_dates=True)

In [ ]:
bakery.info()

In [ ]:
bakery.head()

# Exploring Time Series

## Select A Product - Bread

In [ ]:
bakery["quantity"] = 1
bakery.head()

In [ ]:
product = bakery[bakery["Item"]=="Bread"]
product.head()

## Resampling Time Series

Pandas has extremely neat support for time series. The .resample() method easily allows to aggregate (upsample) and expand time series (downsample)

- **D** daily level
- **W** weekly level
- **Q** daily level
- **Y** daily level


In [ ]:
product.resample("D").quantity.sum()  # daily

In [ ]:
product.resample("W").quantity.sum()  # weeky

In [ ]:
product.resample("Q").quantity.sum()  # quarterly

## Plot Time Series

In [ ]:
product_ts = product.resample("D").quantity.sum()

In [ ]:
product_ts.plot()

## Create some Time Series Features

In [ ]:
product_df = pd.DataFrame(product_ts)
product_df.head()

### Add "day of the week" feature

In [ ]:
product_df["day_of_week"] = product_df.index.weekday
product_df

In pandas the week starts with 0 (Mondays) and ends with 6 (Sunday). Our data is going from Tuesday to Tuesday.

We have data for 23 weeks and 1 day (in total 162 days). We drop the first day such that we have data on 23 full weeks (eases the handling of time series a bit).

Dropping first instance:

In [ ]:
product_df = product_df.iloc[1: , :]
product_df

### Add "week of year" Feature

In [ ]:
product_df.index.isocalendar().week

Due to the Wednesday to Tuesday structure of the data, we have inequally spaced weeks.

But we can shift the weeks by two days such that we have "effective weeks"

In [ ]:
product_df.loc[:, "week_of_year"] = product_df.index.shift(periods = - 2, freq = "D").isocalendar().week.values
product_df

### Add "is_closed" feature

In [ ]:
product_df.loc[:, "is_closed"] = (product_df["quantity"]==0).astype(int)
product_df

## Seasonal Decomposition

a standarf plotting function:

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomp = seasonal_decompose(product_df["quantity"])
decomp.plot()

# Create Test & Train Data

In [ ]:
test = product_df[-28:] # get last 28 days
train = product_df[0:-28] # get data until first day of testing day

# Naive Forecasting

Import required modules

In [ ]:
import numpy as np
from sklearn.metrics import root_mean_squared_error

Make copy of training data "history" for walk forward validation

In [ ]:
history = train
history

Initialize empty list for performance metrics in each week of the 4 weeks

In [ ]:
performance_collector = []

In [ ]:
for w in test["week_of_year"].unique():
  y_pred = [history.tail(1)["quantity"]]*7 # get value of last day of last week in history
  actual = test[test["week_of_year"]==w]["quantity"] # get actual values to be predicted in week w

  history = pd.concat([history, test[test["week_of_year"]==w]]) # append training data to history; we shift one week forward in next iteration
  performance_collector.append(root_mean_squared_error(actual,y_pred)) #calculalate error in week w

In [ ]:
print(performance_collector)
print(np.mean(performance_collector))

In [ ]:
plot_week_NAIVE = pd.DataFrame({"NAIVE":performance_collector})
plot_week_NAIVE

In [ ]:
def performance_week_plot(list_of_predictions):
  import matplotlib.pyplot as plt
  pd.concat(list_of_predictions, axis=1).plot(color=["#7FC97F","#BEAED4","#FDC086","#FFFF99"], alpha=0.5, linewidth=5)
  plt.ylabel("RMSE")
  plt.xlabel("Week")
  plt.xticks(ticks=[0,1,2,3],labels=["1","2","3","4"])
  plt.title("Performance by Week")
  plt.show()

In [ ]:
performance_week_plot([plot_week_NAIVE])

In [ ]:
def performance_avg_plot(list_of_predictions):
  import matplotlib.pyplot as plt
  pd.concat(list_of_predictions, axis=1).mean().plot(kind="bar",color=["#7FC97F","#BEAED4","#FDC086","#FFFF99"], alpha=0.6)
  plt.ylabel("RMSE")
  plt.xlabel("Approach")
  plt.xticks(rotation=0)
  plt.title("4 Weeks Average")
  plt.show()

In [ ]:
performance_avg_plot([plot_week_NAIVE])

# SNAIVE Foreacasting

## Initialize 4 Week-Forward Validation

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
history = train
performance_collector = []

## 4 Week-Forward Validation

In [ ]:
for w in test["week_of_year"].unique():
  y_pred = history.tail(7)["quantity"]
  actual = test[test["week_of_year"]==w]["quantity"]

  history = pd.concat([history, test[test["week_of_year"]==w]])

  performance_collector.append(root_mean_squared_error(actual,y_pred))

In [ ]:
print(performance_collector)
print(np.mean(performance_collector))

In [ ]:
plot_week_SNAIVE = pd.DataFrame({"SNAIVE":performance_collector})

In [ ]:
performance_week_plot([plot_week_NAIVE, plot_week_SNAIVE])

In [ ]:
performance_avg_plot([plot_week_NAIVE, plot_week_SNAIVE])

# ARIMA Forecasting

## Plot ACF

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
acf = plot_acf(product_ts, lags=40,color='g')

*  Correlation of Time Series with lagged version of itself
*  We see strong seasonal patterns: lags at day 7, 14, 21 are correlated with our sales >> there is a "weekly sales memory"
*   To some extend also the day before


## Plot PACF

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
pacf = plot_pacf(product_ts, lags=40,color='g')

*  Pacf are somilar to acf, but pacf reduce the effect of prior lags
*  Pacf can be used to decide how many lags to include
*  We can see that the "weekly sales memory" becomes weaker over time, after week 3 (day 21) past sales does not contain no usable information any more


## Initialize 4 Week-Forward Validation

pmdarima is a very handy package for ARIMA, but it is not installed within the anaconda framework and google collab

*   In Jupyter: the following command must be executed once
*   In google collab: the following command must be executed everytime pmdarima should be used



In [ ]:
!pip install pandas==2.2.0
!pip install pmdarima


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima
history = train
performance_collector = []

## Train Auto ARIMA Model

In [ ]:
arima = auto_arima(train["quantity"], error_action='ignore', trace=True,
                   suppress_warnings=True, maxiter=30,
                   seasonal=True, m=7)

## 4 Week-Forward Validation

In [ ]:
for w in test["week_of_year"].unique():
  y_pred = pd.Series(arima.predict(7))
  actual = test[test["week_of_year"]==w]["quantity"]

  history = pd.concat([history, test[test["week_of_year"]==w]])
  arima.update(test[test["week_of_year"]==w]["quantity"])

  performance_collector.append(root_mean_squared_error(actual,y_pred))

In [ ]:
print(performance_collector)
print(np.mean(performance_collector))

In [ ]:
plot_week_ARIMA = pd.DataFrame({"ARIMA":performance_collector})

In [ ]:
performance_week_plot([plot_week_NAIVE, plot_week_SNAIVE,plot_week_ARIMA])

In [ ]:
performance_avg_plot([plot_week_NAIVE, plot_week_SNAIVE,plot_week_ARIMA])

# Going Machine Learning

## Initial Feature Engineering

Manually Define time lags. We have seen in the acf and pacf plots that going up to 2 weeks back in past might yield important information. So, we take day 7 to 14.

In [ ]:
shifts = np.arange(7, 15).astype(int)
shifts

The following codes creates the shifted variables that we defined in **shifts**.

In [ ]:
train

In [ ]:
shifted_data = {"lag_{}_day".format(day_shift): train["quantity"].shift(day_shift) for day_shift in shifts}
shifted_data = pd.DataFrame(shifted_data)
shifted_data.head(20)

We merge the shifted variables to our training data

In [ ]:
train = pd.concat([train,shifted_data],axis=1)
train

Also we can use the seasonal component of the seasonal decomposition. We see that this patterns repeats all 7 days

In [ ]:
decomp.seasonal.plot()

In [ ]:
train["seasonality"] = decomp.seasonal
train

Delete NA from training data

In [ ]:
train = train.dropna()

Separate Feature and Label

In [ ]:
X_train = train.drop("quantity", axis=1)
y_train = train["quantity"]

## Train Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forecast_forest = RandomForestRegressor(n_estimators=10000)
forecast_forest.fit(X_train, y_train)

## Initialze 4 Week-Forward Validation

In [ ]:
import numpy as np
from sklearn.metrics import root_mean_squared_error
history = train
performance_collector = []

## 4 Week-Forward Validation

In [ ]:
for w in test["week_of_year"].unique():

  dat = history.tail(14) # get last 14 days of available data

  dat = pd.concat([dat,test[test["week_of_year"]==w]]) #add test data

  shifted_data = {"lag_{}_day".format(day_shift): dat["quantity"].shift(day_shift) for day_shift in shifts} # lag training data
  shifted_data = pd.DataFrame(shifted_data)

  dat= pd.concat([dat[["quantity","day_of_week","week_of_year","is_closed"]],shifted_data],axis=1) # add shifted data

  dat = dat.dropna() # delete missing data

  dat["seasonality"] = decomp.seasonal.head(7).values #add seasonal data
  pred_dat = dat.drop("quantity", axis=1)

  y_pred = forecast_forest.predict(pred_dat)
  actual = test[test["week_of_year"]==w]["quantity"]

  performance_collector.append(root_mean_squared_error(actual,y_pred))

  history = pd.concat([history, dat])

In [ ]:
print(performance_collector)
print(np.mean(performance_collector))

In [ ]:
plot_week_RF = pd.DataFrame({"RF":performance_collector})

In [ ]:
performance_week_plot([plot_week_NAIVE, plot_week_SNAIVE,plot_week_ARIMA,plot_week_RF])

In [ ]:
performance_avg_plot([plot_week_NAIVE, plot_week_SNAIVE,plot_week_ARIMA,plot_week_RF])